### Install & import

In [ ]:
%pip install langchain
%pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.7/823.7 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.agents import create_csv_agent, create_pandas_dataframe_agent
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory

### OpenAI login

In [ ]:
import getpass
pswd = getpass.getpass('OpenAI API key:')

OpenAI API key:··········


### Load CSV files

In [ ]:
# template.csv
from google.colab import files
upload = files.upload()

Saving template.csv to template.csv


In [ ]:
# table_A.csv
from google.colab import files
upload = files.upload()

Saving table_A.csv to table_A.csv


In [ ]:
# llm = OpenAI(temperature=0, openai_api_key=pswd, model_name="davinci")
llm = OpenAI(temperature=0, openai_api_key=pswd, model_name="text-davinci-003")

### *Part 1* Map table A column to template's columns and save resulting DF

In [ ]:
memory = ConversationBufferMemory()
agent = create_csv_agent(llm, 'table_A.csv', verbose=True)
agent.memory = memory

In [ ]:
mapping_to_template = agent.run(f"Load template_df with template_df = pd.read_csv('template.csv'). \
Compute sequence similarity between all possible pairs of columns from two dataframes. \
Pair columns when ratios when ratio is positive.")
mapping_to_template



> Entering new AgentExecutor chain...

Thought: I need to compare the columns of two dataframes.
Action: python_repl_ast
Action Input: from difflib import SequenceMatcher
Observation: 
Thought: I need to compare the columns of two dataframes.
Action: python_repl_ast
Action Input: template_df = pd.read_csv('template.csv')
Observation: NameError("name 'pd' is not defined")
Thought: I need to import pandas
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought: I need to read the csv file
Action: python_repl_ast
Action Input: template_df = pd.read_csv('template.csv')
Observation: 
Thought: I need to compare the columns of two dataframes.
Action: python_repl_ast
Action Input: for col1 in df.columns:
    for col2 in template_df.columns:
        ratio = SequenceMatcher(None, df[col1], template_df[col2]).ratio()
        if ratio > 0:
            print(f'{col1} and {col2} have a similarity ratio of {ratio}')
Observation: FullName and EmployeeName have a similarity ra

'The columns FullName and EmployeeName, Monthly_Premium and Premium, Full_Name and EmployeeName, Insurance_Type and Plan, and Monthly_Cost and Premium have a similarity ratio of 1.0.'

In [ ]:
mapping_json = agent.run(f"{mapping_to_template}. Use this info to create JSON with column mapping. Print this JSON").replace('\'','"')
mapping_json



> Entering new AgentExecutor chain...
Thought: I need to create a JSON with the column mapping
Action: python_repl_ast
Action Input: import json
              df_columns = df.columns.tolist()
              mapping = {
                  'FullName': 'EmployeeName',
                  'Monthly_Premium': 'Premium',
                  'Full_Name': 'EmployeeName',
                  'Insurance_Type': 'Plan',
                  'Monthly_Cost': 'Premium'
              }
              print(json.dumps(mapping))
Observation: IndentationError: unexpected indent (<unknown>, line 2)
Thought: I need to fix the indentation
Action: python_repl_ast
Action Input: import json
              df_columns = df.columns.tolist()
              mapping = {
                  'FullName': 'EmployeeName',
                  'Monthly_Premium': 'Premium',
                  'Full_Name': 'EmployeeName',
                  'Insurance_Type': 'Plan',
                  'Monthly_Cost': 'Premium'
              }
              prin

'{"FullName": "EmployeeName", "Monthly_Premium": "Premium", "Full_Name": "EmployeeName", "Insurance_Type": "Plan", "Monthly_Cost": "Premium"}'

In [ ]:
import json
template_df = pd.read_csv('template.csv')
not_matched = list(template_df.columns.difference(json.loads(mapping_json).values()))
not_matched

['Date', 'PolicyNumber']

In [ ]:
# mapping_missing = agent.run(f"Find columns in df best matching to {not_matched}. Print as map from df to new columns as a JSON").replace('\'','"')
matched_to = agent.run(f"Find columns in df best matching to {not_matched}. Print as a list").replace('\'','"')
mapping_missing = dict(zip(json.loads(matched_to), not_matched))
mapping_missing



> Entering new AgentExecutor chain...
Thought: I need to find the columns that match the given list
Action: python_repl_ast
Action Input: df.columns.tolist()
Observation: ['Date_of_Policy', 'FullName', 'Insurance_Plan', 'Policy_No', 'Monthly_Premium', 'Department', 'JobTitle', 'Policy_Start', 'Full_Name', 'Insurance_Type', 'Policy_Num', 'Monthly_Cost']
Thought: I need to filter the list to only include the columns I need
Action: python_repl_ast
Action Input: [col for col in df.columns.tolist() if col in ['Date', 'PolicyNumber']]
Observation: []
Thought: I need to use the correct column names
Action: python_repl_ast
Action Input: [col for col in df.columns.tolist() if col in ['Date_of_Policy', 'Policy_No']]
Observation: ['Date_of_Policy', 'Policy_No']
Thought: I now know the final answer
Final Answer: ['Date_of_Policy', 'Policy_No']

> Finished chain.


{'Date_of_Policy': 'Date', 'Policy_No': 'PolicyNumber'}

In [ ]:
_columns = json.loads(mapping_json)
_columns.update(mapping_missing)
_columns

{'FullName': 'EmployeeName',
 'Monthly_Premium': 'Premium',
 'Full_Name': 'EmployeeName',
 'Insurance_Type': 'Plan',
 'Monthly_Cost': 'Premium',
 'Date_of_Policy': 'Date',
 'Policy_No': 'PolicyNumber'}

In [ ]:
# apply mapping_code to `table_A.csv`
df = pd.read_csv("table_A.csv")
df.rename(columns=_columns, inplace=True)
df = df[pd.read_csv("template.csv").columns]
# Keep the first occurrence of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,2023-05-01,John Doe,Gold,AB-12345,150.0
1,2023-05-02,Jane Smith,Silver,CD-67890,100.0
2,2023-05-03,Michael Brown,Bronze,EF-10111,50.0
3,2023-05-04,Alice Johnson,Gold,GH-12121,150.0
4,2023-05-05,Bob Wilson,Silver,IJ-13131,100.0
5,2023-05-06,Carol Martinez,Bronze,KL-14141,50.0
6,2023-05-07,David Anderson,Gold,MN-15151,150.0
7,2023-05-08,Eva Thomas,Silver,OP-16161,100.0
8,2023-05-09,Frank Jackson,Bronze,QR-17171,50.0
9,2023-05-10,Grace White,Gold,ST-18181,150.0


### *Part 2* Align the column formatting of the DF with that of template_df

In [ ]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True)
agent.memory = memory

In [ ]:
template_df = pd.read_csv('template.csv')
col_transform_code = agent.run(f"Make the formatting of every columns match the correcponding column in {template_df}. Output the code")
col_transform_code



> Entering new AgentExecutor chain...

Thought: I need to change the format of the date column
Action: python_repl_ast
Action Input: df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
Observation: 
Thought: I need to change the format of the PolicyNumber column
Action: python_repl_ast
Action Input: df['PolicyNumber'] = df['PolicyNumber'].str.replace('-', '')
Observation: 
Thought: I now know the final answer
Final Answer: df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
df['PolicyNumber'] = df['PolicyNumber'].str.replace('-', '')

> Finished chain.


"df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')\ndf['PolicyNumber'] = df['PolicyNumber'].str.replace('-', '')"

In [ ]:
# apply mapping_code and col_transform_code to `table_A.csv`
df = pd.read_csv("table_A.csv")
df.rename(columns=_columns, inplace=True)
df = df[pd.read_csv("template.csv").columns]
# Keep the first occurrence of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df['Date'] = pd.to_datetime(df['Date'])
exec(col_transform_code)
df

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,01-05-2023,John Doe,Gold,AB12345,150.0
1,02-05-2023,Jane Smith,Silver,CD67890,100.0
2,03-05-2023,Michael Brown,Bronze,EF10111,50.0
3,04-05-2023,Alice Johnson,Gold,GH12121,150.0
4,05-05-2023,Bob Wilson,Silver,IJ13131,100.0
5,06-05-2023,Carol Martinez,Bronze,KL14141,50.0
6,07-05-2023,David Anderson,Gold,MN15151,150.0
7,08-05-2023,Eva Thomas,Silver,OP16161,100.0
8,09-05-2023,Frank Jackson,Bronze,QR17171,50.0
9,10-05-2023,Grace White,Gold,ST18181,150.0


In [ ]:
agent.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content="Load template_df with template_df = pd.read_csv('template.csv'). Compute sequence similarity between all possible pairs of columns from two dataframes. Pair columns when ratios when ratio is positive.", additional_kwargs={}, example=False), AIMessage(content='The columns FullName and EmployeeName, Monthly_Premium and Premium, Full_Name and EmployeeName, Insurance_Type and Plan, and Monthly_Cost and Premium have a similarity ratio of 1.0.', additional_kwargs={}, example=False), HumanMessage(content='The columns FullName and EmployeeName, Monthly_Premium and Premium, Full_Name and EmployeeName, Insurance_Type and Plan, and Monthly_Cost and Premium have a similarity ratio of 1.0.. Use this info to create JSON with column mapping. Print this JSON', additional_kwargs={}, example=False), AIMessage(content='{"FullName": "EmployeeName", "Monthly_Premium": "Premium", "Full_Name": "EmployeeName", "Insurance_T

In [ ]:
from langchain.schema import HumanMessage, AIMessage
prompt =[]
completion = []
for message1, message2 in zip(agent.memory.chat_memory.messages, agent.memory.chat_memory.messages[1:]):
  if type(message1) == HumanMessage: # we start with prompt always
    print(type(message1), type(message2))
    prompt.append(message1.content)
    completion.append(message2.content)
df = pd.DataFrame(zip(prompt, completion), columns = ['prompt','completion'])
df

<class 'langchain.schema.HumanMessage'> <class 'langchain.schema.AIMessage'>
<class 'langchain.schema.HumanMessage'> <class 'langchain.schema.AIMessage'>
<class 'langchain.schema.HumanMessage'> <class 'langchain.schema.AIMessage'>
<class 'langchain.schema.HumanMessage'> <class 'langchain.schema.AIMessage'>


,prompt,completion
0,Load template_df with template_df = pd.read_cs...,"The columns FullName and EmployeeName, Monthly..."
1,"The columns FullName and EmployeeName, Monthly...","{""FullName"": ""EmployeeName"", ""Monthly_Premium""..."
2,"Find columns in df best matching to ['Date', '...","['Date_of_Policy', 'Policy_No']"
3,Make the formatting of every columns match the...,df['Date'] = df['Date'].dt.strftime('%d-%m-%Y'...


In [ ]:
df.to_json("examples.jsonl", orient='records', lines=True)

In [ ]:
!openai tools fine_tunes.prepare_data -f examples.jsonl -q

Analyzing...

- Your file contains 4 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whi

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = pswd

In [37]:
!openai api fine_tunes.create -t "examples_prepared.jsonl" -m "davinci"

Found potentially duplicated files with name 'examples_prepared.jsonl', purpose 'fine-tune' and size 1947 bytes
file-fAfFLDOCmxVXiRMinUxzqH0e
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: 
Upload progress: 100% 1.95k/1.95k [00:00<00:00, 2.21Mit/s]
Uploaded file from examples_prepared.jsonl: file-B1zdsGeMxZAQqGfRMOtAKXDf
Created fine-tune: ft-9kf23uQSoOxWLqVfIqwxcfeQ
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-05-15 03:35:17] Created fine-tune: ft-9kf23uQSoOxWLqVfIqwxcfeQ
[2023-05-15 03:35:49] Fine-tune costs $0.08
[2023-05-15 03:35:50] Fine-tune enqueued. Queue number: 0
[2023-05-15 03:35:54] Fine-tune started



In [40]:
!openai api fine_tunes.get -i ft-9kf23uQSoOxWLqVfIqwxcfeQ

{
  "created_at": 1684121717,
  "events": [
    {
      "created_at": 1684121717,
      "level": "info",
      "message": "Created fine-tune: ft-9kf23uQSoOxWLqVfIqwxcfeQ",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1684121749,
      "level": "info",
      "message": "Fine-tune costs $0.08",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1684121750,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1684121754,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1684121872,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1684121874,
      "level": "info",
      "message": "Completed epoch 2/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1684121876,
      "level": "info",
